In [1]:
import pandas as pd
import collections as c
from collections import Counter
import re
import csv

import nltk as nltk
import nltk.corpus  
from nltk.text import Text
import sys
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('Food_Inspections.csv')

In [3]:
df.head(5)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2484567,ROYALTY,ROYALTY,1306130.0,Restaurant,Risk 1 (High),3810 W 63RD ST,CHICAGO,IL,60629.0,02/05/2021,Non-Inspection,No Entry,NaN,41.778837,-87.718361,"(-87.71836138998039, 41.778836516734856)"
1,2473041,ROSATI'S GRANT PARK,ROSATI'S,2762683.0,Restaurant,Risk 1 (High),23 E ADAMS ST,CHICAGO,IL,60603.0,01/22/2021,License,Pass,NaN,41.879391,-87.626848,"(-87.62684825563626, 41.879391313239694)"
2,2463667,M & M FOOD,M & M FOOD,1193207.0,NaN,Risk 3 (Low),7073 N WESTERN AVE,CHICAGO,IL,60645.0,12/11/2020,Canvass,Out of Business,NaN,42.010769,-87.690067,"(-87.69006746560828, 42.01076851593425)"
3,2457076,SUBWAY,SUBWAY,2411733.0,Restaurant,Risk 1 (High),4406 S PULASKI RD,CHICAGO,IL,60632.0,11/27/2020,Canvass,Pass,NaN,41.813101,-87.723928,"(-87.72392832288558, 41.81310105112748)"
4,2456274,TRATTORIA ULTIMO,TRATTORIA ULTIMO,2069755.0,Restaurant,Risk 1 (High),1953 W CHICAGO AVE,CHICAGO,IL,60622.0,11/05/2020,Canvass,Out of Business,NaN,41.895808,-87.676752,"(-87.67675213462381, 41.89580774819584)"


In [4]:
Results = df['Results']

In [5]:
Results.unique()

array(['No Entry', 'Pass', 'Out of Business', 'Not Ready', 'Fail',
       'Pass w/ Conditions', 'Business Not Located'], dtype=object)

In [6]:
Fail = df["Results"] == "Fail"
Failed = df.loc[Fail]


In [7]:
Failed.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
27,2373427,THE BUNKER,THE BUNKER,2718517.0,Restaurant,Risk 3 (Low),1515 W MONROE ST,CHICAGO,IL,60607.0,06/09/2020,License,Fail,NaN,41.880086,-87.665054,"(-87.66505441566734, 41.88008566659281)"
35,2370354,SLIM'S,SLIM'S,2622236.0,Restaurant,Risk 1 (High),2020 W MONTROSE AVE,CHICAGO,IL,60618.0,05/08/2020,Complaint,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.961556,-87.679862,"(-87.67986227331792, 41.96155579127668)"
41,2369855,Chicago Prep Master Tenant LLC,Chicago Prep Master Tenant,2723671.0,Shared Kitchen,Risk 1 (High),5801 W Dickens AVE,CHICAGO,IL,60639.0,04/28/2020,License,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.917869,-87.770697,"(-87.77069733431132, 41.91786924215764)"
48,2369400,DSD DELI,DSD DELI,2626178.0,DELI/GROCERY,Risk 1 (High),5205 N MILWAUKEE,CHICAGO,IL,60630.0,04/17/2020,License,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.974849,-87.766829,"(-87.76682905043768, 41.97484853157327)"
49,2369394,AMSTAR,AMSTAR,2152845.0,Grocery Store,Risk 2 (Medium),8950-58 S ASHLAND AVE,CHICAGO,IL,60620.0,04/17/2020,Canvass Re-Inspection,Fail,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.730593,-87.663058,"(-87.66305762988213, 41.730593381254266)"


In [8]:
Failed = Failed.dropna(subset=['Violations'])

In [9]:
Failed = Failed[Failed['Violations'].astype(bool)] 

In [10]:
Failed['Violations'].isnull()


35        False
41        False
48        False
49        False
59        False
60        False
64        False
148       False
153       False
160       False
164       False
165       False
166       False
211       False
217       False
220       False
242       False
248       False
259       False
269       False
299       False
310       False
312       False
320       False
321       False
332       False
334       False
338       False
342       False
345       False
          ...  
218067    False
218071    False
218077    False
218078    False
218079    False
218081    False
218083    False
218087    False
218091    False
218101    False
218106    False
218111    False
218123    False
218126    False
218128    False
218134    False
218148    False
218149    False
218150    False
218154    False
218157    False
218161    False
218162    False
218169    False
218170    False
218172    False
218176    False
218180    False
218187    False
218191    False
Name: Violations, Length

In [11]:
violation_master_list = []
f = lambda x: x['Violations'].split("|")
violation_list = Failed.apply(f, axis=1)

In [12]:
flat_list = [item for sublist in violation_list for item in sublist]

In [13]:
x = []
pattern = r"[0-9]+\.(.*?)(?=Comments:)"

for i in flat_list:
    result = re.findall(pattern,str(i)) #perform regex & identify/extract the violation code only
    x.append(result)#append the violation code into violation_master_list


In [14]:
#print(reg_list)

In [14]:
#reg_list = map(str, x)
reg_list = [str(i) for i in x]
#reg_list

In [ ]:
#reg_list = tuple(tuple(p) for p in x)

#for p in x:
 #   reg_list = str(p) 

In [15]:
#words = nltk.tokenize.word_tokenize(reg_list)
words = []

for i in reg_list:
    res = word_tokenize(i)
    words.append(res)
    


In [16]:
#fdist = nltk.FreqDist(words)
fdist = []

for i in words:
    res = nltk.FreqDist(i)
    fdist.append(res)

In [17]:
print(fdist[1:10])

[FreqDist({"'": 2, '[': 1, 'CITY': 1, 'OF': 1, 'CHICAGO': 1, 'FOOD': 1, 'SERVICE': 1, 'SANITATION': 1, 'CERTIFICATE': 1, '-': 1, ...}), FreqDist({"'": 2, ',': 2, 'EMPLOYEE': 2, 'AND': 2, '[': 1, 'MANAGEMENT': 1, 'FOOD': 1, 'CONDITIONAL': 1, ';': 1, 'KNOWLEDGE': 1, ...}), FreqDist({"'": 2, '[': 1, 'PROCEDURES': 1, 'FOR': 1, 'RESPONDING': 1, 'TO': 1, 'VOMITING': 1, 'AND': 1, 'DIARRHEAL': 1, 'EVENTS': 1, ...}), FreqDist({"'": 2, '[': 1, 'NO': 1, 'BARE': 1, 'HAND': 1, 'CONTACT': 1, 'WITH': 1, 'RTE': 1, 'FOOD': 1, 'OR': 1, ...}), FreqDist({"'": 2, '[': 1, 'ADEQUATE': 1, 'HANDWASHING': 1, 'SINKS': 1, 'PROPERLY': 1, 'SUPPLIED': 1, 'AND': 1, 'ACCESSIBLE': 1, '-': 1, ...}), FreqDist({"'": 2, '[': 1, 'FOOD': 1, 'SEPARATED': 1, 'AND': 1, 'PROTECTED': 1, '-': 1, ']': 1}), FreqDist({"'": 2, '[': 1, 'PROPER': 1, 'HOT': 1, 'HOLDING': 1, 'TEMPERATURES': 1, '-': 1, ']': 1}), FreqDist({"'": 2, '[': 1, 'PROPER': 1, 'DATE': 1, 'MARKING': 1, 'AND': 1, 'DISPOSITION': 1, '-': 1, ']': 1}), FreqDist({"'": 2, '

In [18]:
word_list = [item for sublist in fdist for item in sublist]

In [19]:
word_list[1:10]
type(word_list)

list

In [45]:
#x1 = tuple(tuple(p) for p in x)
RawTokens=Counter(word_list).most_common(10) 
RawTokens

[('[', 252655),
 (']', 252655),
 ("'", 252286),
 ('-', 252286),
 (',', 168300),
 ('AND', 119831),
 (':', 86802),
 ('MAINTAINED', 76294),
 ('FOOD', 71184),
 ('EQUIPMENT', 63367)]

In [59]:
import gc

gc.collect()

0

In [22]:
stopwords = set(nltk.corpus.stopwords.words('english'))
words = [word for word in word_list if len(word) > 1]

In [23]:
#words

In [24]:
# Remove stopwords
words = [word for word in words if word not in stopwords]

#for word in words:
    #if word not in stopwords:
        #words.append(word)

In [25]:
# Remove punctuation
words = [word for word in words if word.isalpha()]

#for word in words:
 #   if word.isalpha():
  #      words.append(word)

In [26]:
words

['PERSON',
 'IN',
 'CHARGE',
 'PRESENT',
 'DEMONSTRATES',
 'KNOWLEDGE',
 'AND',
 'PERFORMS',
 'DUTIES',
 'CITY',
 'OF',
 'CHICAGO',
 'FOOD',
 'SERVICE',
 'SANITATION',
 'CERTIFICATE',
 'MANAGEMENT',
 'FOOD',
 'EMPLOYEE',
 'AND',
 'CONDITIONAL',
 'KNOWLEDGE',
 'RESPONSIBILITIES',
 'REPORTING',
 'PROCEDURES',
 'FOR',
 'RESPONDING',
 'TO',
 'VOMITING',
 'AND',
 'DIARRHEAL',
 'EVENTS',
 'NO',
 'BARE',
 'HAND',
 'CONTACT',
 'WITH',
 'RTE',
 'FOOD',
 'OR',
 'ALTERNATIVE',
 'PROCEDURE',
 'PROPERLY',
 'ALLOWED',
 'ADEQUATE',
 'HANDWASHING',
 'SINKS',
 'PROPERLY',
 'SUPPLIED',
 'AND',
 'ACCESSIBLE',
 'FOOD',
 'SEPARATED',
 'AND',
 'PROTECTED',
 'PROPER',
 'HOT',
 'HOLDING',
 'TEMPERATURES',
 'PROPER',
 'DATE',
 'MARKING',
 'AND',
 'DISPOSITION',
 'THERMOMETERS',
 'PROVIDED',
 'ACCURATE',
 'FOOD',
 'PROPERLY',
 'LABELED',
 'ORIGINAL',
 'CONTAINER',
 'INSECTS',
 'RODENTS',
 'ANIMALS',
 'NOT',
 'PRESENT',
 'PERSONAL',
 'CLEANLINESS',
 'WIPING',
 'CLOTHS',
 'PROPERLY',
 'USED',
 'STORED',
 'WAREWAS

In [30]:
fdist_cleaned = nltk.FreqDist(words)

fdist_cleaned


FreqDist({'AND': 119831, 'MAINTAINED': 76294, 'FOOD': 71184, 'EQUIPMENT': 63367, 'CONSTRUCTED': 61555, 'OF': 57806, 'CLEAN': 57779, 'PROPERLY': 56371, 'INSTALLED': 54391, 'CLEANING': 48220, ...})

In [46]:
##5
#no5 = fdist.most_common(10)

#c_word_list = [item for sublist in fdist_cleaned for item in sublist]

CleanedTokens = Counter(fdist_cleaned).most_common(10)
CleanedTokens

[('AND', 119831),
 ('MAINTAINED', 76294),
 ('FOOD', 71184),
 ('EQUIPMENT', 63367),
 ('CONSTRUCTED', 61555),
 ('OF', 57806),
 ('CLEAN', 57779),
 ('PROPERLY', 56371),
 ('INSTALLED', 54391),
 ('CLEANING', 48220)]

In [34]:
porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()

In [35]:
words_l = (words[0:50])
words_s = ', '.join(words_l)
type (words_s)

str

In [68]:
words_s

'PERSON, IN, CHARGE, PRESENT, DEMONSTRATES, KNOWLEDGE, AND, PERFORMS, DUTIES, CITY, OF, CHICAGO, FOOD, SERVICE, SANITATION, CERTIFICATE, MANAGEMENT, FOOD, EMPLOYEE, AND, CONDITIONAL, KNOWLEDGE, RESPONSIBILITIES, REPORTING, PROCEDURES, FOR, RESPONDING, TO, VOMITING, AND, DIARRHEAL, EVENTS, NO, BARE, HAND, CONTACT, WITH, RTE, FOOD, OR, ALTERNATIVE, PROCEDURE, PROPERLY, ALLOWED, ADEQUATE, HANDWASHING, SINKS, PROPERLY, SUPPLIED, AND'

In [69]:
print([porter.stem(t) for t in words[0:50]])

['person', 'IN', 'charg', 'present', 'demonstr', 'knowledg', 'and', 'perform', 'duti', 'citi', 'OF', 'chicago', 'food', 'servic', 'sanit', 'certif', 'manag', 'food', 'employe', 'and', 'condit', 'knowledg', 'respons', 'report', 'procedur', 'for', 'respond', 'TO', 'vomit', 'and', 'diarrheal', 'event', 'NO', 'bare', 'hand', 'contact', 'with', 'rte', 'food', 'OR', 'altern', 'procedur', 'properli', 'allow', 'adequ', 'handwash', 'sink', 'properli', 'suppli', 'and']


In [36]:
Porter =[porter.stem(t) for t in words[0:]]
Porter_tokens = nltk.FreqDist(Porter)



[('clean', 129356),
 ('and', 119831),
 ('maintain', 79842),
 ('food', 76438),
 ('equip', 63367),
 ('construct', 61555),
 ('OF', 57806),
 ('properli', 56371),
 ('instal', 54391),
 ('surfac', 41634)]

In [57]:
PorterTokens = Porter_tokens.most_common(10)
PorterTokens

[('clean', 129356),
 ('and', 119831),
 ('maintain', 79842),
 ('food', 76438),
 ('equip', 63367),
 ('construct', 61555),
 ('OF', 57806),
 ('properli', 56371),
 ('instal', 54391),
 ('surfac', 41634)]

In [58]:
Lancaster = [lancaster.stem(t) for t in words[0:]]
lancaster_tokens = nltk.FreqDist(Lancaster)


In [48]:
LancasterTokens = lancaster_tokens.most_common(10)
LancasterTokens

[('cle', 133920),
 ('and', 119831),
 ('maintain', 79842),
 ('food', 76438),
 ('prop', 68613),
 ('equip', 63367),
 ('construct', 61555),
 ('of', 57806),
 ('instal', 54391),
 ('surfac', 41634)]

In [72]:
print([lancaster.stem(t) for t in words[0:50]])

['person', 'in', 'charg', 'pres', 'demonst', 'knowledg', 'and', 'perform', 'duty', 'city', 'of', 'chicago', 'food', 'serv', 'sanit', 'cert', 'man', 'food', 'employ', 'and', 'condit', 'knowledg', 'respons', 'report', 'proc', 'for', 'respond', 'to', 'vomit', 'and', 'diarrh', 'ev', 'no', 'bar', 'hand', 'contact', 'with', 'rte', 'food', 'or', 'altern', 'proc', 'prop', 'allow', 'adequ', 'handwash', 'sink', 'prop', 'supply', 'and']


In [73]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\manya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
wnl = nltk.WordNetLemmatizer()

In [74]:
print([wnl.lemmatize(t) for t in words[0:50]])

['PERSON', 'IN', 'CHARGE', 'PRESENT', 'DEMONSTRATES', 'KNOWLEDGE', 'AND', 'PERFORMS', 'DUTIES', 'CITY', 'OF', 'CHICAGO', 'FOOD', 'SERVICE', 'SANITATION', 'CERTIFICATE', 'MANAGEMENT', 'FOOD', 'EMPLOYEE', 'AND', 'CONDITIONAL', 'KNOWLEDGE', 'RESPONSIBILITIES', 'REPORTING', 'PROCEDURES', 'FOR', 'RESPONDING', 'TO', 'VOMITING', 'AND', 'DIARRHEAL', 'EVENTS', 'NO', 'BARE', 'HAND', 'CONTACT', 'WITH', 'RTE', 'FOOD', 'OR', 'ALTERNATIVE', 'PROCEDURE', 'PROPERLY', 'ALLOWED', 'ADEQUATE', 'HANDWASHING', 'SINKS', 'PROPERLY', 'SUPPLIED', 'AND']


In [43]:
Lemmatize =[wnl.lemmatize(t) for t in words[0:]]
Lemmatize_tokens = nltk.FreqDist(Lemmatize)



In [49]:
LemmatizeTokens = Lemmatize_tokens.most_common(10)
LemmatizeTokens 

[('AND', 119831),
 ('MAINTAINED', 76294),
 ('FOOD', 71184),
 ('EQUIPMENT', 63367),
 ('CONSTRUCTED', 61555),
 ('OF', 57806),
 ('CLEAN', 57779),
 ('PROPERLY', 56371),
 ('INSTALLED', 54391),
 ('CLEANING', 48220)]

In [59]:
print ('RawTokens:', RawTokens)

print ('CleanedTokens:', CleanedTokens)

print ('PorterTokens:', PorterTokens)

print ('LancasterTokens:', LancasterTokens)

print ('LemmatizeTokens:', LemmatizeTokens)

RawTokens: [('[', 252655), (']', 252655), ("'", 252286), ('-', 252286), (',', 168300), ('AND', 119831), (':', 86802), ('MAINTAINED', 76294), ('FOOD', 71184), ('EQUIPMENT', 63367)]
CleanedTokens: [('AND', 119831), ('MAINTAINED', 76294), ('FOOD', 71184), ('EQUIPMENT', 63367), ('CONSTRUCTED', 61555), ('OF', 57806), ('CLEAN', 57779), ('PROPERLY', 56371), ('INSTALLED', 54391), ('CLEANING', 48220)]
PorterTokens: [('clean', 129356), ('and', 119831), ('maintain', 79842), ('food', 76438), ('equip', 63367), ('construct', 61555), ('OF', 57806), ('properli', 56371), ('instal', 54391), ('surfac', 41634)]
LancasterTokens: [('cle', 133920), ('and', 119831), ('maintain', 79842), ('food', 76438), ('prop', 68613), ('equip', 63367), ('construct', 61555), ('of', 57806), ('instal', 54391), ('surfac', 41634)]
LemmatizeTokens: [('AND', 119831), ('MAINTAINED', 76294), ('FOOD', 71184), ('EQUIPMENT', 63367), ('CONSTRUCTED', 61555), ('OF', 57806), ('CLEAN', 57779), ('PROPERLY', 56371), ('INSTALLED', 54391), ('CL

It looks like the best results were given either by a basic cleaning of the data or from the Lemmatizer. The other two methods, Lancaster and Porter, cut off too much of each token, while not really producing results that would be useful for increasing the likelihood of identifying alternate forms of common words.